# **CA13**: EJERCICIO 1 || PROCESOS Y LECTURA

In [1]:
#include <sys/types.h>
#include <unistd.h> // Para funciones POSIX como fork() y getpid()
#include <sys/wait.h>
#include <sys/mman.h> // Para memoria compartida y mmap
#include <fcntl.h> // Para banderas de memoria compartida como O_CREAT y O_RDWR
#include <cstring>
#include <vector>
#include <fstream> // Para escritura en archivos
#include <cstdlib> // Para srand() y rand()
#include <ctime> // Para la semilla de srand() usando time(NULL)

//Las bibliotecas nos permitirán trabajar con funciones de sistema que 
//son necesarias para crear y gestionar procesos, así como para utilizar 
//la memoria compartida y realizar operaciones de entrada y salida.

//Definición de la memoria compartida que se utilizará como identificador cuando se cree y
//acceda a esta.
#define SHM_NAME "/shared_mem"

// Función que genera un array de enteros aleatorios.
void generateRandomArray(int *array, int size);

In [2]:
void generateRandomArray(int *array, int size) {
    srand(time(NULL) + getpid());  // Semilla única por proceso hijo, configurada utilizando
    //time(NULL) + getpid(), asegurando que cada proceso hijo tenga una semilla distinta
    for (int i = 0; i < size; ++i) {
        array[i] = rand() % 100;  // Genera números aleatorios entre 0 y 99
    }
}

In [3]:
int main() {
    const int N = 3;              // Número de procesos hijos
    int sizes[N] = {5, 7, 6};     // Tamaños de arrays para cada hijo
    int pipes[N][2];              // Pipes para comunicación entre procesos
    pid_t pids[N];                // Almacenará los PID de los procesos hijos

    // 1. Crear pipes para cada hijo
    for (int i = 0; i < N; ++i) {
        if (pipe(pipes[i]) == -1) {
            perror("Error creando el pipe");
            return 1;
        }
    }

    // 2. Crear memoria compartida para almacenar los arrays generados
    int shm_fd = shm_open(SHM_NAME, O_CREAT | O_RDWR, 0666);
    if (shm_fd == -1) {
        perror("Error creando la memoria compartida");
        return 1;
    }

    int totalSize = 0;
    for (int i = 0; i < N; ++i) totalSize += sizes[i];
    if (ftruncate(shm_fd, totalSize * sizeof(int)) == -1) {
        perror("Error ajustando tamaño de la memoria compartida");
        return 1;
    }

    int *shared_mem = (int *)mmap(0, totalSize * sizeof(int), PROT_WRITE | PROT_READ, MAP_SHARED, shm_fd, 0);
    if (shared_mem == MAP_FAILED) {
        perror("Error mapeando la memoria compartida");
        return 1;
    }

    // 3. Crear procesos hijos
    int offset = 0;
    for (int i = 0; i < N; ++i) {
        pids[i] = fork();
        if (pids[i] == 0) { // Código del proceso hijo
            close(pipes[i][0]); // Cerrar extremo de lectura del pipe

            int size = sizes[i];
            int *array = shared_mem + offset;
            generateRandomArray(array, size); // Generar array aleatorio

            // Calcular la suma dentro del hijo
            int sum = 0;
            for (int j = 0; j < size; ++j) {
                sum += array[j];
            }

            // Enviar la suma al proceso padre
            write(pipes[i][1], &sum, sizeof(sum));
            close(pipes[i][1]); // Cerrar extremo de escritura

            munmap(shared_mem, totalSize * sizeof(int)); // Desmapear memoria
            exit(0); // Terminar proceso hijo
        } else if (pids[i] > 0) { // Código del proceso padre
            close(pipes[i][1]); // Cerrar extremo de escritura del pipe
            offset += sizes[i];
        } else {
            perror("Error al crear el proceso hijo");
            return 1;
        }
    }

    // 4. Esperar a que los hijos terminen
    for (int i = 0; i < N; ++i) {
        waitpid(pids[i], nullptr, 0);
    }

    // 5. Leer los datos de los hijos y mostrar por pantalla
    offset = 0;
    for (int i = 0; i < N; ++i) {
        int sum;
        read(pipes[i][0], &sum, sizeof(sum));
        close(pipes[i][0]); // Cerrar extremo de lectura del pipe

        // Mostrar array y suma por pantalla
        printf("Array %d: ", i + 1);
        for (int j = 0; j < sizes[i]; ++j) {
            printf("%d ", shared_mem[offset + j]);
        }
        printf("| Sum: %d\n", sum);

        offset += sizes[i];
    }

    // 6. Liberar memoria compartida
    munmap(shared_mem, totalSize * sizeof(int));
    shm_unlink(SHM_NAME);

    return 0;
}

main();

Array 1: 3 51 83 94 59 | Sum: 290
Array 2: 60 0 47 81 85 49 37 | Sum: 359
Array 3: 39 63 61 68 98 96 | Sum: 425


# Diferencias con el primer modelo

### Reducir el tamaño de N = 5 a N = 3
`const int N = 3;              // Número de procesos hijos`</br>
`int sizes[N] = {5, 7, 6};`</br>
### Calculamos y enviamos la suma del array.

### Mostrar los resultados por pantalla en lugar de archivo

**Código anterior:**

**Código actualizado:**

### Eliminar el manejo de archivos

- **Antes:** El archivo arrays.txt se creaba y se cerraba tras escribir los resultados
- **Después:** Todo el código relacionado con la apertura y cierre del archivo fue eliminado

### Cambios en el formato de la Impresión

**Código anterior:**

**Código actualizado:**

## Resumen de Cambios
**Reducción de Procesos y Tamaños de Arrays:**

- Se simplificó el tamaño de los datos procesados.

**Cálculo y Envío de la Suma:**

- La suma ahora se calcula en el proceso hijo y se envía al padre.

**Impresión en Pantalla:**

- Los resultados ahora se imprimen con printf en lugar de ser escritos en un archivo.

**Eliminación del Archivo:**

Se eliminó toda la lógica relacionada con std::ofstream.
Estos cambios simplifican el programa y hacen que sea más fácil de seguir y probar.

## CUALES SON LAS VENTAJAS DE UTILIZAR HILOS SOBRE PROCESOS?

1. ***Más rápido:*** La creación de hilos es menos costosa que la de procesos.
2. ***Menos complejidad:*** No es necesario usar *pipes* ni *shm_open()*, ya que los hilos comparten memoria de manera natural.